# Generate Signatures for Tables

## MSTICPYConfig

In [1]:
%%writefile msticpyconfig.yaml
AzureSentinel:
  Workspaces:
    ASIHuntOMSWorkspaceV4:
      TenantId: 72f988bf-86f1-41af-91ab-2d7cd011db47
      WorkspaceId: 52b1ab41-869e-4138-9e40-2a4457f09bf0
    CyberSecuritySoc:
      TenantId: 72f988bf-86f1-41af-91ab-2d7cd011db47
      WorkspaceId: 8ecf8077-cf51-4820-aadd-14040956f35d
    Default:
      TenantId: 72f988bf-86f1-41af-91ab-2d7cd011db47
      WorkspaceId: 8ecf8077-cf51-4820-aadd-14040956f35d

Overwriting msticpyconfig.yaml


## Initiializing MSTICPy

In [2]:
# Core MSTICPy initialization for Notebooks
from msticpy.nbtools import nbinit
nbinit.init_notebook(namespace=globals());

# Load query providers (typically you'll be using just one)
qry_prov = QueryProvider("AzureSentinel")

Please wait. Loading Kqlmagic extension...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Connect to workspace 
```
WorkspaceConfig(workspace=WS_NAME)
```

By default, uses the Default entry

In [3]:
qry_prov.connect(WorkspaceConfig())

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Read in Tables to be Analyzed

In [4]:
import os

with open('tables.txt') as file:
    tables = file.read().splitlines()

# Create folder if it doesn't exist
    
for table in tables:
    if not os.path.isdir('./data/' + table):
        os.makedirs('./data/' + table)

In [5]:
# tableName contains the name of the table we are querying (string)
# maxAge is the oldest result we want to retrieve in weeks minimum 1 week(int)

def queryTable(tableName, maxAge=1):

    assert(maxAge >= 1)
    df = pd.DataFrame()
    
    for weeks in range(0, maxAge*7, 7):
        print("Retrieving data for {tableName} from {week} week(s) ago".format(tableName = tableName, week = int(weeks/7) + 1))
        queryString = "{tableName} | where TimeGenerated between (ago({end}d) .. ago({start}d))".format(tableName = tableName, start = weeks, end = weeks + 7)
        queryResult = qry_prov.exec_query(queryString)
        df = pd.concat([df, queryResult])
    return df

In [6]:
# Query N and Pickle N weeks worth of data
# Generates and saves raw data from queries
# Takes in a list of tables, a boolean that determines if we are drawing from a random sample
# and an integer n is either the number of weeks if sample is false or n is the number of smamples

def generateData(tables, sample=True, n=52):
    for table in tables:
        
        # If we are not sampling, use the queryTable to pull n weeks and concatenate that
        # else pull a random sample of n rows
        
        df = pd.DataFrame()
        
        if sample == False:
            for weeks in range(0, maxAge*7, 7):
                print("Retrieving data for {tableName} from {week} week(s) ago".format(tableName = tableName, week = int(n/7) + 1))
                queryString = "{tableName} | where TimeGenerated between (ago({end}d) .. ago({start}d))".format(tableName = tableName, start = n, end = n + 7)
                queryResult = qry_prov.exec_query(queryString)
                df = pd.concat([df, queryResult])
        else:
            print("Retrieving a sample of {nSamples} rows from {tableName}.".format(tableName = table, nSamples = n))
            queryString = "{tableName} | sample {nSamples}".format(tableName = table, nSamples = n)
            df = qry_prov.exec_query(queryString)
        df.to_pickle('./data/{table}/raw.pkl'.format(table = table))
        print("{table} pickld and saved in ./data/{table}/raw.pkl\n".format(table = table))

## Sample and Pickle 10,000 Rows

In [7]:
generateData(tables, True, 10000)

Retrieving a sample of 10000 rows from OfficeActivity.


<IPython.core.display.Javascript object>

OfficeActivity pickld and saved in ./data/OfficeActivity/raw.pkl

Retrieving a sample of 10000 rows from SigninLogs.


<IPython.core.display.Javascript object>

SigninLogs pickld and saved in ./data/SigninLogs/raw.pkl

Retrieving a sample of 10000 rows from SecurityEvent.


<IPython.core.display.Javascript object>

SecurityEvent pickld and saved in ./data/SecurityEvent/raw.pkl

Retrieving a sample of 10000 rows from CommonSecurityLog.


<IPython.core.display.Javascript object>

CommonSecurityLog pickld and saved in ./data/CommonSecurityLog/raw.pkl



## Read Data from Pickle

In [8]:
import pandas as pd
import numpy as np

rawData = {}
for table in tables:
    rawData[table] = pd.read_pickle("data/{tableName}/raw.pkl".format(tableName=table))

## Clean Tables

In [37]:
import datetime
import re

# Contain exact matches you want to filter out of table
exactMatches = []

# Contains regular expressions you want to filter out of table
regexes = [
    r'^.*[Tt][Ii][Mm][Ee].*$', # Regex for checking if the word time (case insensitive) is in the string. No ignorecase flag
]

cleanData = {}

for table in rawData:

    print('Cleaning table {table}'.format(table=table))
    
    # Remove unhashable types such as lists or dictionaries and convert them to a string
    cleanTable = rawData[table].applymap(lambda x: str(x) if isinstance(x, list) or isinstance(x, dict) or isinstance(x, datetime.datetime) else x)
    
    # Remove features that may be continuous values (i.e. time) using regular expressions and exact matches
    
    for feature in cleanTable:
        
        # Check if this feature is included in our exactMatches to remove
        
        if feature in exactMatches:
            cleanTable = cleanTable.drop([feature], axis=1)
            print('{feature} due to being an exact match'.format(feature=feature))
            continue
        
        # Check if the values in a column are all the same, if so we remove
        
        if cleanTable[feature].nunique() <= 1:
            print('{feature} due to being an invariant column.'.format(feature=feature))
            cleanTable = cleanTable.drop([feature], axis=1)
            continue
            
        # Else compare to regular expressions
            
        for regex in regexes:
            if re.match(regex, feature):
                cleanTable = cleanTable.drop([feature], axis=1)
                print('{feature} due to being a match with regular expression: {regex}'.format(feature=feature, regex=regex))
                break

    # Finds empty columns to prevent them from being dropped
    emptyCol = []
    for column in cleanTable:
        # Convert to numpy
        data = cleanTable[column].to_numpy() 
        if (data[0] == np.nan or data[0] == '') and (data[0] == data).all():
            emptyCol.append(column)
            
    # Copy columns over to be added back after duplicates are removed
    col = cleanTable[emptyCol]
    
    # Transpose the cleaned table and drop duplicate rows. Re-transpose to get back to the original table
    cleanTable = cleanTable.T.drop_duplicates().T
    
    # Add empty columns back into table and reorder
    cleanTable = pd.concat([cleanTable, col], axis=1)

    # Save cleaned data in a pickled file using
    cleanTable.to_pickle('./data/{table}/cleaned.pkl'.format(table = table))

    cleanData[table] = cleanTable
    
cleanData['OfficeActivity']

Cleaning table OfficeActivity
ChannelType due to being an invariant column.
ExtraProperties due to being an invariant column.
AddonName due to being an invariant column.
TabType due to being an invariant column.
OldValue due to being an invariant column.
ChatThreadId due to being an invariant column.
ChatName due to being an invariant column.
AppDistributionMode due to being an invariant column.
TargetUserId due to being an invariant column.
OperationScope due to being an invariant column.
AzureADAppId due to being an invariant column.
Type due to being an invariant column.
_ResourceId due to being an invariant column.
Cleaning table SigninLogs
TenantId due to being an invariant column.
SourceSystem due to being an invariant column.
ResourceId due to being an invariant column.
OperationName due to being an invariant column.
OperationVersion due to being an invariant column.
Category due to being an invariant column.
ResultSignature due to being an invariant column.
DurationMs due to be

,UserAgent,RecordType,Operation,UserType,UserKey,OfficeWorkload,ResultStatus,OfficeObjectId,UserId,ClientIP,Site_,ItemType,EventSource,MachineId,Site_Url,SourceRelativeUrl,SourceFileName,SourceFileExtension,Event_Data,Parameters,ExternalAccess,OriginatingServer,OrganizationName,Logon_Type,InternalLogonType,...,Folder,DestFolder,Folders,AffectedItems,Item,ModifiedProperties,OfficeId,TargetUserOrGroupName,TargetUserOrGroupType,MessageId,Members,TeamName,TeamGuid,ChannelName,ChannelGuid,AddOnType,Name,NewValue,ItemName,CommunicationType,AADGroupId,AddOnGuid,OperationProperties,AppId,ClientAppId
0,,50,MailItemsAccessed,Regular,100320003F88D275,Exchange,Succeeded,,seb@seccxpninja.onmicrosoft.com,,,,,,,,,,,,False,DM5PR06MB3180 (15.20.4200.000)\r\n,seccxpninja.onmicrosoft.com,Owner,0.0,...,,,"[{""FolderItems"":[{""InternetMessageId"":""<DM5PR06MB31807DC0A3BE68202EC27F06E6E29@DM5PR06MB3180.nam...",,,,d8d7a5dc-5eb4-4afc-b014-4478701be04a,,,,None,,,,,,,,,,,,"[{'Name': 'MailAccessType', 'Value': 'Bind'}, {'Name': 'IsThrottled', 'Value': 'False'}]",7ab7862c-4c57-491e-8a45-d52a7e023983,
1,,50,MailItemsAccessed,Regular,100320003F88D275,Exchange,Succeeded,,seb@seccxpninja.onmicrosoft.com,,,,,,,,,,,,False,DM5PR06MB3180 (15.20.4200.000)\r\n,seccxpninja.onmicrosoft.com,Owner,0.0,...,,,"[{""FolderItems"":[{""InternetMessageId"":""<DM5PR06MB31802786FC9F33E6C31F4952E6E19@DM5PR06MB3180.nam...",,,,20219ec7-b669-4df1-8b98-fb040e266469,,,,None,,,,,,,,,,,,"[{'Name': 'MailAccessType', 'Value': 'Bind'}, {'Name': 'IsThrottled', 'Value': 'False'}]",7a5fbd1c-3e6d-461a-9075-83049393b3a7,7a5fbd1c-3e6d-461a-9075-83049393b3a7
2,,50,MailItemsAccessed,Regular,100320003F88D275,Exchange,Succeeded,,seb@seccxpninja.onmicrosoft.com,,,,,,,,,,,,False,DM5PR06MB3180 (15.20.4200.000)\r\n,seccxpninja.onmicrosoft.com,Owner,0.0,...,,,"[{""FolderItems"":[{""InternetMessageId"":""<DM5PR06MB31807DC0A3BE68202EC27F06E6E29@DM5PR06MB3180.nam...",,,,508e896a-ebb2-4667-946f-cd2a5044253e,,,,None,,,,,,,,,,,,"[{'Name': 'MailAccessType', 'Value': 'Bind'}, {'Name': 'IsThrottled', 'Value': 'False'}]",7a5fbd1c-3e6d-461a-9075-83049393b3a7,7a5fbd1c-3e6d-461a-9075-83049393b3a7
3,,50,MailItemsAccessed,Regular,10032000EC025FA0,Exchange,Succeeded,,AzureDevSub@seccxpninja.onmicrosoft.com,,,,,,,,,,,,False,BN0P221MB0446 (15.20.4200.000)\r\n,seccxpninja.onmicrosoft.com,Owner,0.0,...,,,"[{""FolderItems"":[{""InternetMessageId"":""<BYAPR06MB56531B10203EB268790352C1AFE19@BYAPR06MB5653.nam...",,,,b0e36006-6db3-4b08-902f-6d451a86cddf,,,,None,,,,,,,,,,,,"[{'Name': 'MailAccessType', 'Value': 'Bind'}, {'Name': 'IsThrottled', 'Value': 'False'}]",7a5fbd1c-3e6d-461a-9075-83049393b3a7,7a5fbd1c-3e6d-461a-9075-83049393b3a7
4,,50,MailItemsAccessed,Regular,1003200160C8B403,Exchange,Succeeded,,KDickens@seccxp.ninja,,,,,,,,,,,,False,BL3P221MB0387 (15.20.4200.000)\r\n,seccxpninja.onmicrosoft.com,Owner,0.0,...,,,"[{""FolderItems"":[{""InternetMessageId"":""<MN2PR21MB1519EA5AFDAC9603BCF1F409E1E19@MN2PR21MB1519.nam...",,,,81896c9f-c6dc-41c0-b416-0e7f58fe7b69,,,,None,,,,,,,,,,,,"[{'Name': 'MailAccessType', 'Value': 'Bind'}, {'Name': 'IsThrottled', 'Value': 'False'}]",5a2ee4c5-13b8-465b-88d7-75ecf16830ad,3c8e478f-21ca-493a-b87c-c7366d664d54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,,ExchangeAdmin,Set-User,DcAdmin,NT AUTHORITY\SYSTEM (Microsoft.Exchange.Management.ForwardSync),Exchange,True,NAMPR06A007.PROD.OUTLOOK.COM/Microsoft Exchange Hosted Organizations/seccxpninja.onmicrosoft.com...,NT AUTHORITY\SYSTEM (Microsoft.Exchange.Management.ForwardSync),,,,,,,,,,,"[\r\n {\r\n ""Name"": ""Identity"",\r\n ""Value"": ""4b2462a4-bbee-495a-a0e1-f23ae524cc9c\\515d0...",True,BN7PR06MB4114 (15.20.4287.033),seccxpninja.onmicrosoft.com,,NaN,...,,,,,,,98470b8c-7518-4e95-0c2a-08d93ef7b5da,,,,None,,,,,,,,,,,,None,,
9996,,ExchangeItem,Send,Regular,10032001582C649E,Exchange,

## Binarize Data

In [38]:
binData = {}

for table in cleanData:
    
    # Replace empty cells with NaN 
    # Comment this line out if you do not want to remove empty strings
    binTable = cleanData[table].replace(r'^\s*$', np.nan, regex=True)

    # Replace NaN values with 0 and all others with 1
    binTable = binTable.notnull().astype('int')

    # Save binary data in a pickled file
    binTable.to_pickle('./data/{table}/binarized.pkl'.format(table = table))
    
    # Clean continuous values
    
    binData[table] = binTable  

binData['OfficeActivity']

,UserAgent,RecordType,Operation,UserType,UserKey,OfficeWorkload,ResultStatus,OfficeObjectId,UserId,ClientIP,Site_,ItemType,EventSource,MachineId,Site_Url,SourceRelativeUrl,SourceFileName,SourceFileExtension,Event_Data,Parameters,ExternalAccess,OriginatingServer,OrganizationName,Logon_Type,InternalLogonType,...,Folder,DestFolder,Folders,AffectedItems,Item,ModifiedProperties,OfficeId,TargetUserOrGroupName,TargetUserOrGroupType,MessageId,Members,TeamName,TeamGuid,ChannelName,ChannelGuid,AddOnType,Name,NewValue,ItemName,CommunicationType,AADGroupId,AddOnGuid,OperationProperties,AppId,ClientAppId
0,0,1,1,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,...,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
1,0,1,1,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,...,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1
2,0,1,1,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,...,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1
3,0,1,1,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,...,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1
4,0,1,1,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,...,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9996,0,1,1,1,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,...,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9997,0,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9998,0,1,1,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,...,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0


In [39]:
# Takes in a string of 0 and 1 as the signature
# Takes in a list of column names
# Returns a list of features that are present and features that are missing

def getPresentColumns(signature, columns):
    present = []
    missing = []
    for index in range(len(signature)):
        if int(signature[index]):
            present.append(columns[index])
        else:
            missing.append(columns[index])
    return present, missing

In [40]:
# Counts the number of times a datapoint shows up in the features
# For example, it counts how many times the IP 44.150.161.58 shows up in the clientIP column

def countTypes(row, columns, presentFeatures, featureDict):
    for index in range(len(row)):
        
        currentFeature = columns[index]
        value = row[index]
        
        # If the feature is missing we won't count it
        if currentFeature not in presentFeatures:
            continue
            
        if value not in featureDict[currentFeature]:
            featureDict[currentFeature][value] = 1
        else:
            featureDict[currentFeature][value] += 1
    return featureDict

## Generate Dictionary of Signatures

In [41]:
import json

tableSignatures = {}
for table in binData:
    print('Generating dictionary for {tableName}'.format(tableName = table))
    columns = binData[table].columns
    signatureDict = {}
    
    for index, row in binData[table].iterrows():
        signature = ''.join(map(str, row.values.tolist()))
        
        # If this signature does not exist
        if signature not in signatureDict:
            
            # Identify Present/Missing features
            present, missing = getPresentColumns(signature, columns)
            # Generate and update number of different data types in the feature dictionary
            featureDict = {i: {} for i in present}
            featureDict = countTypes(cleanData[table].iloc[index], columns, present, featureDict)
            
            signatureDict[signature] = {
                'count': 1,
                'presentFeatures': present,
                'missingFeatures': missing,
                'featureDict': featureDict
            }
        else:
            signatureDict[signature]['count'] += 1
            signatureDict[signature]['featureDict'] = countTypes(cleanData[table].iloc[index], columns, signatureDict[signature]['presentFeatures'], signatureDict[signature]['featureDict'])
            
    tableSignatures[table] = signatureDict
    with open('./data/{table}/signatureDictionary.json'.format(table=table), 'w') as f:
        json.dump(signatureDict, f)

Generating dictionary for OfficeActivity
Generating dictionary for SigninLogs
Generating dictionary for SecurityEvent
Generating dictionary for CommonSecurityLog


In [43]:
from pprint import pprint

print(signatureDict[signature])

{'count': 2040, 'presentFeatures': ['DeviceVendor', 'DeviceProduct', 'DeviceEventClassID', 'LogSeverity', 'DeviceAction', 'CommunicationDirection', 'DestinationIP', 'SourceIP', 'DeviceVersion', 'Activity', 'ApplicationProtocol', 'DestinationServiceName', 'DestinationHostName', 'FileType', 'ReceivedBytes', 'SentBytes', 'RequestClientApplication', 'RequestContext', 'RequestMethod', 'SourceTranslatedAddress', 'SourceUserPrivileges', 'SourceUserName', 'DeviceCustomNumber1', 'DeviceCustomNumber1Label', 'DeviceCustomString1', 'DeviceCustomString1Label', 'DeviceCustomString2', 'DeviceCustomString2Label', 'DeviceCustomString3', 'DeviceCustomString3Label', 'DeviceCustomString4', 'DeviceCustomString4Label', 'DeviceCustomString5', 'DeviceCustomString5Label', 'DeviceCustomString6', 'DeviceCustomString6Label', 'AdditionalExtensions', '_ResourceId'], 'missingFeatures': ['DeviceFacility', 'DestinationPort', 'DeviceAddress', 'DeviceName', 'Message', 'Protocol', 'SourcePort', 'DestinationTranslatedAddr